## План

1. S modeling

2. T modeling

3. X learner

4. R learner

In [1]:
from pathlib import Path
import sys
 
if sys.argv:
    sys.path.insert(0, str(Path('/Users/ogrobertino/UpliftModelingResearch/').resolve()))

In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from src.distributions_check import check_feature_distributions_by_stat_test, check_feature_distributions_by_model
from src.datasets import sample_features, TorchDataset, NumpyDataset

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.weightstats import ztest

from sklearn.model_selection import train_test_split

from tqdm.auto import tqdm

pd.options.display.max_rows = 200
pd.options.display.max_colwidth = 200
from sklearn.metrics import roc_auc_score

def local_read(path):
    return pd.DataFrame(yt.read_table(path))

import lightgbm as lgb
import causalml
import causalml.metrics as cmetrics
import causalml.inference.tree as ctree
import causalml.inference.meta.tlearner as tlearner
import causalml.inference.meta.slearner as slearner
from causalml.inference.tree import UpliftTreeClassifier, UpliftRandomForestClassifier
# import causalml.inference.meta.rlearner as rlearner
# import causalml.inference.meta.xlearner as xlearner
# import causalml.inference.meta.drlearner as drlearner

Failed to import duecredit due to No module named 'duecredit'


In [4]:
features_percent = 100

path_to_data_train = f'../data/lazada/{features_percent}/train.tsv'
path_to_data_test = f'../data/lazada/{features_percent}/test.tsv'
train = NumpyDataset(path_to_data_train)
test = NumpyDataset(path_to_data_test)

In [5]:
col_treatment = "treatment"
col_target = "target"
cols_features = train.data.drop(["treatment", 'target'], axis=1).columns

In [6]:
def apply(model, data, cols_features, col_treatment, col_target):
    scores = data.copy(deep=True)
    scores['score'] = model.predict(scores.loc[:, cols_features])
    return scores[['score', col_treatment, col_target]]

In [7]:
train.data = train.data.sample(n=10000)

In [ ]:
tmodel = tlearner.BaseTClassifier(
    learner=lgb.LGBMClassifier(),
    control_learner=lgb.LGBMClassifier(),
    treatment_learner=lgb.LGBMClassifier(),
    control_name=0
)

tmodel.fit(
    X=train.data.loc[:, cols_features].values,
    treatment=train.data.loc[:, col_treatment].values,
    y=train.data.loc[:, col_target].values,
)

In [1]:
pred_tmodel = apply(tmodel, oot, cols_features, col_treatment, col_target)

NameError: name 'apply' is not defined

In [2]:
cmetrics.auuc_score(
    pred_tmodel, 
    outcome_col=col_target, 
    treatment_col=col_treatment, 
)

NameError: name 'cmetrics' is not defined